In [2]:
from scapy.all import Ether, ARP, IP, EAPOL, Raw, srp1, Ether
import threading
import time
from dataclasses import dataclass

%load_ext ipyext

ModuleNotFoundError: No module named 'ipyext'

# Virtual switch

Your code will receive ethernet "frames" -- sequences of bytes representing packets received by a network interface. Your task is to implement a virtual switch that will forward these frames along but first it will tra

In [ ]:
# MAC table: a dictionary that maps MAC addresses to port numbers
self.mac_table: dict[str, int] = {}
# ARP table: a dictionary that maps IP addresses to MAC addresses
self.arp_table: dict[str, str] = {}
# A set of all ports that have been authenticated
self.authenticated_ports = set()

# Representing ether frames

Make a class called `Frame` that will represent an ethernet frame. It should have the following attributes:

- `src` - a string representing the source MAC address
- `dst` - a string representing the destination MAC address
- `data` - a list of integers representing the data in the frame

In [ ]:
@dataclass
class Frame:
    pass

# Construct an ether frame from raw bytes

%%

In [ ]:
def simulate_traffic(switch):
    # Simulate incoming frames
    for _ in range(3):
        # Simulate an Ethernet frame
        eth_frame = Ether(src="00:00:00:00:00:01", dst="00:00:00:00:00:02") / IP(src="192.168.1.1", dst="192.168.1.2") / Raw(load="Hello, Virtual Switch!")
        src_port = 1
        switch.forward_frame(bytes(eth_frame), src_port)
        
        # Simulate an ARP request
        arp_request = Ether(src="00:00:00:00:00:01", dst="ff:ff:ff:ff:ff:ff") / ARP(hwsrc="00:00:00:00:00:01", psrc="192.168.1.1", hwdst="00:00:00:00:00:00", pdst="192.168.1.2")
        switch.handle_arp(arp_request)
        
        # Simulate 802.1X authentication
        eapol_start = Ether(src="00:00:00:00:00:03", dst="01:80:c2:00:00:03", type=ETH_P_PAE) / EAPOL(type=1)
        switch.handle_eapol(eapol_start, 3)
        
        time.sleep(5)  # Wait for 5 seconds before sending the next frame

In [ ]:
Ether()

In [ ]:
# define a `learn_mac` method that updates the MAC table
    def learn_mac(self, src_mac: str, in_port: int):
        self.mac_table[src_mac] = in_port`

In [5]:
class VirtualSwitch:
    def __init__(self):
        self.mac_table = {}
        self.arp_table = {}
        self.authenticated_ports = set()

    def learn_mac(self, mac, port):
        self.mac_table[mac] = port

    def forward_frame(self, frame, src_port):
        ether = Ether(frame)
        
        self.learn_mac(ether.src, src_port)
        
        if ether.dst in self.mac_table:
            dst_port = self.mac_table[ether.dst]
            if dst_port != src_port:
                self.send_frame(frame, dst_port)
        else:
            self.flood_frame(frame, src_port)

    def flood_frame(self, frame, src_port):
        for port in self.authenticated_ports:
            if port != src_port:
                self.send_frame(frame, port)

    def send_frame(self, frame, port):
        # In a real implementation, this would send the frame to the physical port
        print(f"Sending frame to port {port}: {Ether(frame).summary()}")

    def handle_arp(self, frame):
        arp = frame[ARP]
        
        if arp.op == 1:  # ARP Request
            self.arp_table[arp.psrc] = arp.hwsrc
            
            if arp.pdst in self.arp_table:
                self.send_arp_reply(arp.hwsrc, arp.psrc, self.arp_table[arp.pdst], arp.pdst)

    def send_arp_reply(self, target_mac, target_ip, sender_mac, sender_ip):
        arp_reply = Ether(dst=target_mac, src=sender_mac) / ARP(
            op=2,
            hwsrc=sender_mac,
            psrc=sender_ip,
            hwdst=target_mac,
            pdst=target_ip
        )
        
        self.forward_frame(bytes(arp_reply), self.mac_table[sender_mac])

    def authenticate_port(self, port, username, password):
        # Simplified 802.1X authentication
        if username == "user" and password == "password":
            self.authenticated_ports.add(port)
            return True
        return False

    def handle_eapol(self, frame, port):
        eapol = frame[EAPOL]
        
        # Simplified 802.1X authentication process
        if eapol.type == 0x01:  # EAP Start
            # Send EAP Request Identity
            eap_request = Ether(src=frame[Ether].dst, dst=frame[Ether].src) / EAPOL() / Raw(load=b'\x01\x00\x00\x05\x01')
            self.send_frame(bytes(eap_request), port)
        elif eapol.type == 0x00 and len(eapol.payload) > 0:  # EAP packet
            eap = eapol.payload
            if eap.code == 2:  # Response
                if eap.type == 1:  # Identity
                    # Send EAP Request MD5 Challenge (simplified)
                    eap_md5_challenge = Ether(src=frame[Ether].dst, dst=frame[Ether].src) / EAPOL() / Raw(load=b'\x01\x00\x00\x16\x02\x02\x00\x16\x04\x10' + b'\x00' * 16)
                    self.send_frame(bytes(eap_md5_challenge), port)
                elif eap.type == 4:  # MD5 Challenge
                    # In a real implementation, verify the MD5 challenge response
                    # For this example, we'll just accept it
                    eap_success = Ether(src=frame[Ether].dst, dst=frame[Ether].src) / EAPOL() / Raw(load=b'\x01\x00\x00\x04\x03')
                    self.send_frame(bytes(eap_success), port)
                    self.authenticated_ports.add(port)
                    print(f"Port {port} authenticated")



if __name__ == "__main__":
    switch = VirtualSwitch()
    
    # Start simulating traffic
    traffic_thread = threading.Thread(target=simulate_traffic, args=(switch,))
    traffic_thread.start()
    
    # Keep the main thread running
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("Stopping the virtual switch...")

Exception in thread Thread-14 (simulate_traffic):
Traceback (most recent call last):
  File "/Users/syd/miniconda3/envs/infosec_bootcamp/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/Users/syd/miniconda3/envs/infosec_bootcamp/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/syd/miniconda3/envs/infosec_bootcamp/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/dn/6fp4_0814tq2hvvsjcr31mcr0000gn/T/ipykernel_57309/712306832.py", line 94, in simulate_traffic
NameError: name 'ETH_P_PAE' is not defined


Stopping the virtual switch...
